In [ ]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

# parse the document into a list of dictionaries

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# create a knowledge base using the Index method

In [6]:
index = minsearch.Index(
    text_fields = ['question','text','section'], # for performing search 
    keyword_fields = ['course'] # for filtering, ex. to restrict search to specific courses
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents) # uses similar method structure as scikit-learn api

# search with boosting

In [9]:
results = index.search(
    query=q,
    boost_dict={'question':3.0, 'section':0.5}, # boosting determines the importance for the specified fields. ex. question field is 3x more important than other fields
    num_results=5
)

results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

# search with filtering

In [10]:
results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, # filtering restricts the search to specific section. ex. data engineering zoomcampc course
    boost_dict={'question':3.0, 'section':0.5}, 
    num_results=5
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# pass documents to llm to generate answer

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

response.choices[0].message.content

"Typically, enrollment policies for courses vary depending on the institution and the specific course in question. Here are a few steps you can take to determine if you can still enroll:\n\n1. **Check the Course Catalog or Website:** Look up information on the course you’re interested in. Often, universities and other educational institutions will indicate if late enrollment is possible.\n\n2. **Contact the Registrar’s Office:** They usually handle course registrations and can provide clear guidance on whether late enrollment is allowed.\n\n3. **Talk to the Instructor:** Sometimes, instructors have the discretion to allow additional students into their course, even if it's already started.\n\n4. **Check for Open Seats:** If the course has open seats and does not have a strict enrollment cutoff, you might still have a chance to enroll.\n\n5. **Consider the Impact:** Be mindful of that late enrollment might mean you have missed some important early material. Ensure that you are prepared 

In [19]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when asnwering the QUESTION. 
If the CONTEXT does't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
"""

tip: for better results, specify a role for the llm (this is more relevant for advanced llms)

In [16]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [20]:
prompt = prompt_template.format(question=q, context=context)

In [22]:
print(prompt)


You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when asnwering the QUESTION. 
If the CONTEXT does't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on you

In [23]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

# clean the code, make it modular and easier to use

In [24]:
def search(query):
    boost = {'question':3.0, 'section':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'}, 
        boost_dict=boost, 
        num_results=5
    )
    
    return results

In [28]:
results = search('how to run kafka?')

results

[{'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming with kafka',
  'question': 'Module “kafka” not found when trying to run producer.py',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',

In [40]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [35]:
query = 'how to run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)

print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when asnwering the QUESTION. 
    If the CONTEXT does't contain the answer, output NONE.
    
    QUESTION: how to run kafka?
    
    CONTEXT: 
    section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source env/bin/activate
To deactivate it:
deactivate
This works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)
Also the virtual environment should be created only to run the python file. Docker images sho

In [36]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [41]:
query = 'the course just started how do I enroll?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

answer

"To enroll in the course that just started, follow these steps:\n\n1. **Subscribe to the course public Google Calendar**: This is only possible from a desktop device.\n2. **Register before the course starts using this link**: Unfortunately, the specific link is not provided in the context, so you may need to check the course's official website or contact support for the exact registration link.\n3. **Join the course Telegram channel**: This channel will have announcements related to the course.\n4. **Register in DataTalks.Club's Slack and join the channel**: This will help you stay connected with the course community and updates.\n\nMake sure to complete these steps to be fully enrolled and stay updated with the course activities."

In [44]:
def rag(query):
    query = query
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

In [45]:
rag('how do I run kafka?')

'To run Kafka, follow these instructions based on the provided context:\n\n1. **Running a Java Kafka Producer**: \n   Navigate to your project directory and execute the following command:\n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **Running a Python Kafka Producer/Consumer**:\n   - **Create a Virtual Environment**:\n     ```bash\n     python -m venv env\n     source env/bin/activate  # For MacOS/Linux\n     # or\n     env\\Scripts\\activate  # For Windows\n     ```\n\n   - **Install Required Packages**:\n     ```bash\n     pip install -r ../requirements.txt\n     ```\n\n   - **Activate Virtual Environment** (Each time you use it):\n     ```bash\n     source env/bin/activate  # For MacOS/Linux\n     # or\n     env\\Scripts\\activate  # For Windows\n     ```\n\n   - **Deactivate Virtual Environment** (When done):\n     ```bash\n     deactivate\n     ```\n\n   Ensure that all Docker images are up and runnin